In [628]:
# ==============================
# Standard Library Imports
# ==============================
from enum import Enum
import random

# ==============================
# Third-party Library Imports
# ==============================
import matplotlib.pyplot as plt
from IPython.display import display, Latex
from matplotlib.ticker import MultipleLocator
import numpy as np  # Original numpy
import pennylane as qml
import torch 

# PennyLane numpy
from pennylane import numpy as pnp 

In [629]:
# ==============================
# Setup for Quantum Computations
# ==============================

# PennyLane settings
dev = qml.device('default.mixed', wires=1)

# Define Hamiltonian for quantum computations
H = qml.Hamiltonian(coeffs = [-0.5], observables=[qml.PauliZ(0)])

In [630]:
# Global Parameters
Tau_global = 0   # Dephase tau
Paras_global = torch.zeros(2)
Phi_global = torch.zeros(1)
Gamma_ps = 0

def Dephase_factor(tau):
    """ Take tau and return gamma based on the following relation."""
    
    return 1 - np.exp(-2 * tau)

$$
Let, e^{-t/T_2} = e^{-\tau}
$$

$$
\frac{1}{2} 

\begin{bmatrix}

1 & e^{(i\phi - \tau)} \\
e^{(-i\phi - \tau)} & 1

\end{bmatrix}

=

\frac{1}{2} 

\begin{bmatrix}

1 & e^{i\phi} \sqrt{1 - \gamma} \\
e^{-i\phi} \sqrt{1 - \gamma} & 1

\end{bmatrix}$$

$$

Then,
\quad \gamma = 
1 - e^{-2 \tau}
$$

$$ 
e^{-\tau} = \sqrt{1 - \gamma}
$$

In [631]:
@qml.qnode(dev, interface='torch', diff_method='backprop')
def circuit(phi):
    global Paras_global
    theta_x = Paras_global[0]
    phi_z = Paras_global[1]
    
    gamma_dephase_in = Dephase_factor(Tau_global)  

    qml.RX(pnp.pi/2, wires = 0)

    qml.ApproxTimeEvolution(H, phi, 1)
    qml.PhaseDamping(gamma_dephase_in, wires = 0) 

    qml.RZ(phi_z, wires = 0)  # phi_z
    
    qml.RX(theta_x, wires = 0)  # theta_x
    
    return qml.density_matrix(wires = 0)

In [632]:
@qml.qnode(dev, interface='torch', diff_method='backprop')
def Post_selection_Dephase(phi):
    """ Take qnode from circuit_1 and calculate decoherence using kraus operator.
    
    Args:
        phi (float): Phi for Time-approximation. Pass by global variables:'Phi_global'

    Returns:
        qml.density_matrix: Density matrix of full qnode
    """
    
    global Paras_global, Gamma_ps
    
    density_matrix = circuit(phi)
    qml.QubitDensityMatrix(density_matrix, wires = 0)
    
    # Kraus operator for 2*2 matrix
    K = torch.tensor([
        [pnp.sqrt(1 - Gamma_ps), 0],
        [0, 1]
    ], dtype=torch.complex128)
    
    Numerator = K @ density_matrix @ K.conj().T
    Denominator = torch.trace(Numerator)
    rho_ps = Numerator / Denominator

    qml.QubitDensityMatrix(rho_ps, wires = 0)
    
    return qml.density_matrix(wires = 0) 

In [633]:
def Cost_function(paras):
    """ Calculate Classical-Fisher-Information for qnode(=Post_selection_Dephase).
    
    Args:
        paras (Numpy array): [theta_init, tau_1, tau_2, tau_d1, tau_d2, tau_d3]

    Returns:
        _type_: CFI with minus(-) sign.
    """
    
    global Paras_global, Phi_global
    Paras_global = paras

    CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(Phi_global)
    
    return -CFI

In [634]:
# params_t = torch.tensor([0.1,1.8631])
# Phi_global = torch.tensor([np.pi*2])

# Cost_function(params_t)

In [635]:
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# params_t = torch.tensor([np.pi/2, np.pi/3], requires_grad=True, device=device)
# params_t = torch.tensor(params, requires_grad=True, device=torch.device('mps'))

# params_t = params_t.detach().clone().requires_grad_(True)

# params_t = torch.tensor([np.pi/3, np.pi/3], requires_grad=True)
params_t = torch.tensor([1.0, np.pi/2], requires_grad=True)

Phi_global = torch.tensor([np.pi*2])

# opt = torch.optim.LBFGS([params_t])
# opt = torch.optim.Adam([params_t])
opt = torch.optim.LBFGS(
    [params_t], 
    lr=0.01,              # Learning rate
    max_iter=20,          # Maximum number of iterations per optimization step
    max_eval=None,        # Maximum number of function evaluations per optimization step
    tolerance_grad=1e-7,  # Termination tolerance on the gradient norm
    tolerance_change=1e-9,# Termination tolerance on the function value/parameter changes
    history_size=100      # Update history size
)


steps = 500

f_logs = [Cost_function(params_t).item()]
ftol = 1e-10

def closure():
    opt.zero_grad()
    loss = Cost_function(params_t)
    loss.backward()
    return loss

for i in range(steps):
    opt.step(closure)
    fval = Cost_function(opt.param_groups[0]['params'][0]).item()
    print(f"{i+1:03d}th iteration, CFI=", fval)
    f_logs.append(fval)
    if np.abs((fval-f_logs[-2])/fval) < ftol:
        break

001th iteration, CFI= -0.83552180420876
002th iteration, CFI= -0.8979832773906471
003th iteration, CFI= -0.9345417673231093
004th iteration, CFI= -0.9572959364107019
005th iteration, CFI= -0.9718787872703563
006th iteration, CFI= -0.9813773808876088
007th iteration, CFI= -0.9876240742620169
008th iteration, CFI= -0.991756807841371
009th iteration, CFI= -0.994501302687512
010th iteration, CFI= -0.9963286303790702
011th iteration, CFI= -0.9975469868943939
012th iteration, CFI= -0.9983604719496157
013th iteration, CFI= -0.9989038135257027
014th iteration, CFI= -0.9992669972418065
015th iteration, CFI= -0.9995097522925448
016th iteration, CFI= -0.9996720821314171
017th iteration, CFI= -0.9997806669353246
018th iteration, CFI= -0.9998532587482374
019th iteration, CFI= -0.9999018931610861
020th iteration, CFI= -0.9999343167600956
021th iteration, CFI= -0.9999561314625987
022th iteration, CFI= -0.9999706747297972
023th iteration, CFI= -0.9999803305639774
024th iteration, CFI= -0.9999868870211

In [636]:
opt.param_groups[0]

{'params': [tensor([1.5692, 1.5708], requires_grad=True)],
 'lr': 0.01,
 'max_iter': 20,
 'max_eval': 25,
 'tolerance_grad': 1e-07,
 'tolerance_change': 1e-09,
 'history_size': 100,
 'line_search_fn': None}

In [637]:
optimized_param = opt.param_groups[0]['params'][0].detach().numpy()
optimized_param

array([1.5691572, 1.5707964], dtype=float32)

In [638]:
print (torch.__version__) # 1.12.1 이상
print(torch.backends.mps.is_built()) # True 여야 합니다.
print(torch.backends.mps.is_available()) # True 여야 합니다.
!python -c 'import platform;print(platform.platform())'

2.0.1
True
True
macOS-14.1-arm64-arm-64bit


In [639]:
# Data Index
class DataIndex(Enum):
    BEFORE = 0
    PHI = 0
    CFI = 1
    PARAS = 2
    THETA_X = 2
    PHI_Z = 3
    
    
# == BFGS -> Return Data_set:[phi, CFI, 6-Paras] ==
def run_optimization(sweep_data, initial_parameters, gamma_ps, iterations, circuit_select):
    """ 
    Main function to perform optimization over a range of phi values using the BFGS algorithm.
    
    Args:
        sweep_data (tuple): (start, end, step) values for the phi sweep.
        initial_parameters (numpy_array): Initial parameters for optimization.
        gamma_ps (int): Gamma value for post-selection.
        iterations (int): Number of iterations for the optimization.

    Returns:
        numpy.ndarray: A 3-D array containing phi, CFI, and optimized parameters after each iteration.
    """
    
    # Create Data array
    PHI = np.arange(sweep_data[0], sweep_data[1], sweep_data[2])
    Data = np.zeros((iterations + 1, len(PHI), len(initial_parameters) + 2)) 
    Data[:, :, DataIndex.PHI.value] = PHI.squeeze() # Append PHI in to 0th col
    
    # Set global variables
    global Gamma_ps, Phi_global, Paras_global, Tau_global
    Gamma_ps = gamma_ps 
    
    # Declare Paras temp 
    Paras_Temporary = 0
    
    # Store initial CFI data and parameters
    for idx in range(len(PHI)):
        Data[DataIndex.BEFORE.value][idx][DataIndex.CFI.value] = -Cost_function(initial_parameters, circuit_select)
        Data[DataIndex.BEFORE.value][idx][DataIndex.PARAS.value:] = initial_parameters
        
    # Optimize begin
    for iteration in range(1, iterations + 1):
        for phi_idx, phi_current in enumerate(PHI):
            # Determine initial parameters based on the iteration
            if iteration == 1:
                Paras_Temporary = initial_parameters

            else:
                Paras_Temporary = Data[iteration][phi_idx][DataIndex.PARAS.value:]
            
            # Update the global Phi value
            Phi_global = phi_current

            # Determine constraints
            Constraints = get_constraints(phi_current, gamma_ps, Tau_global)

            # Optimize the data
            N = int(phi_current / pnp.pi) * pnp.pi
            if Gamma_ps == 8e-1:
                if Tau_global == 0:
                    Paras_Temporary = pnp.array([pnp.pi/2, pnp.pi/2])

                elif (pnp.pi/2 + N <= phi_current <= 2.1 + N):   # Up next
                    Paras_Temporary = pnp.array([pnp.pi/2, 1])

            Paras_global = Paras_Temporary

            Result_BFGS = BFGS(Paras_Temporary, Constraints, circuit_select)
            Data[iteration][phi_idx][DataIndex.CFI.value] = -Result_BFGS.fun
            Data[iteration][phi_idx][DataIndex.PARAS.value:] = Result_BFGS.x
            
    return Data

def BFGS(initial_parameters, constraints, circuit_select):
    """
    Perform the BFGS optimization algorithm.

    Args:
        initial_parameters (numpy_array): The starting point for the optimization.
        constraints (list of tuple): Bounds on the variables for the optimization.

    Returns:
        OptimizeResult: The result of the optimization process.
    """
    
    optimization_result = torch.optim.LBFGS(
                fun = Cost_function, 
                x0 = initial_parameters, 
                args = (circuit_select,),
                method = 'L-BFGS-B', 
                bounds = constraints,
                jac = gradient,
                hess = hessian,
                tol = 1e-20,
                options={
                    'ftol': 1e-20, 
                    'gtol': 1e-20
                }
            )
    
    
    return optimization_result

def get_constraints(phi_current, gamma_ps, tau_current):
    """
    Calculate the constraints for the optimization based on current phi, gamma and tau values.

    Args:
        phi_current (float): The current value of phi in the optimization loop.
        gamma_ps (float): Gamma value for post-selection.
        tau_current (float): Current value of tau.

    Returns:
        list of tuple: Constraints for the optimization variables.
    """
    
    N = 2*np.pi * int(phi_current / (2*np.pi))
    if Gamma_ps == 8e-1:
        if tau_current == 0:
            return [
                (np.pi/2, np.pi/2),
                (-np.pi/2, 3*np.pi/2)
            ]

        elif tau_current == (5e-2):
            if phi_current < 0.45 + N:
                return [(np.pi/2, np.pi)] * 2
            elif phi_current <= 1.02 + N:
                return [(np.pi/2, 3.70)] * 2
            elif phi_current <= 1.57 + N:
                return [(np.pi/2, 4.24006637)] * 2
            elif 3.00 + N <= phi_current <= 3.67 + N:
                return [(np.pi/2, np.pi/2), (0.32993364, 0.99993333)]
            elif 3.69 + N <= phi_current <= 4.0 + N:
                return [(np.pi/2, np.pi/2), (1.01993369, 1.32993365)]
            elif 4.03 + N <= phi_current <= 4.22 + N:
                return [(np.pi/2, np.pi/2), (1.35993364, 1.54993374)]
            elif 4.24 + N <= phi_current <= 4.69 + N:
                return [(np.pi/2, np.pi/2), (1.56993364, 2.01993363)]
            elif (4.82) + N <= phi_current <= (5.5) + N:
                return [(np.pi/2, np.pi/2), (1.20688106, 1.88688109)]
            elif 5.5 + N <= phi_current <= (6.0) + N:
                return [(np.pi/2, np.pi/2), (1.88688109, 2.38688106)]
            elif 6.05 + N <= phi_current <= (6.15) + N:
                return [(np.pi/2, np.pi/2), (2.43688084, 2.53688103)]

        elif tau_current == 2e-1:
            if phi_current <= 0.5 + N:
                return [(0, np.pi)] * 2
            elif phi_current < 1.58 + N:
                return [(-0.8439553621272445, 3.9939553536152146)] * 2 
            elif phi_current < 2.42 + N:
                return [(-0.8439553621272445, 3.9939553536152146)] * 2 
            elif phi_current < 4 + N:
                return [(0, np.pi/2)] * 2 
            elif phi_current < (1.57 + 3.14) + N:
                return [(np.pi/2, np.pi)] * 2 
        
        elif tau_current == 5e-1:
            return [(-5e-1, pnp.pi + 6e-1 )] * 2
        
        elif 1 <= tau_current <= 4:
            return [(-5e-1, pnp.pi + 35e-2)] * 2